<a href="https://colab.research.google.com/github/utkarsh599/Recomendation-sysy/blob/Abhijeet/learningSpark6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-eu.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

In [58]:
!pip install py4j

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

In [0]:
import findspark
findspark.init()

In [0]:
import pyspark

In [62]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [0]:
comments1 =  spark.read.csv("/content/gdrive/My Drive/Colab Notebooks/GBcomments.csv", inferSchema=True, header=True)

In [0]:
comments2= spark.read.csv("/content/gdrive/My Drive/Colab Notebooks/UScomments.csv", inferSchema=True, header=True)

In [67]:
comments1.show(5)

+-----------+--------------------+-----+-------+
|   video_id|        comment_text|likes|replies|
+-----------+--------------------+-----+-------+
|jt2OHQh0HoQ|It's more accurat...|    0|      0|
|jt2OHQh0HoQ|To be there with ...|    1|      0|
|jt2OHQh0HoQ|Thank gosh, a pla...|    0|      0|
|jt2OHQh0HoQ|What happened to ...|    0|      0|
|jt2OHQh0HoQ|Power is the dise...|    0|      0|
+-----------+--------------------+-----+-------+
only showing top 5 rows



In [68]:
type(comments1)

pyspark.sql.dataframe.DataFrame

In [0]:
from functools import reduce  # For Python 3.x
from pyspark.sql import DataFrame

def unionAll(*dfs):
    return reduce(DataFrame.unionAll, dfs)

comments=unionAll(comments1,comments2)

In [70]:
comments.count()

1410466

In [71]:
comments_rdd = comments.select('comment_text').rdd.flatMap(lambda x: x)
comments_rdd=comments_rdd.filter(lambda x: x is not None).filter(lambda x: x!='')
comments_rdd.take(5)

["It's more accurate to call it the M+ (1000) because the price is closer than calling it the X (10).",
 'To be there with a samsung phone\\n😂😂😂',
 'Thank gosh, a place I can watch it without having to be at HD... my speed doesn’t support HD',
 'What happened to the home button on the iPhone X? *****Cough****copying Samsung******cough',
 'Power is the disease.\xa0 Care is the cure.\xa0 Keep caring for yourself and others as best as you can.\xa0 This is life.']

In [0]:
header = comments_rdd.first()

In [0]:
data_rmv_col = comments_rdd.filter(lambda row: row != header)

In [0]:
def lower(x):
  return x.lower()

In [0]:
lowerCase_sentRDD = data_rmv_col.map(lower)

In [76]:
lowerCase_sentRDD.take(5)

['to be there with a samsung phone\\n😂😂😂',
 'thank gosh, a place i can watch it without having to be at hd... my speed doesn’t support hd',
 'what happened to the home button on the iphone x? *****cough****copying samsung******cough',
 'power is the disease.\xa0 care is the cure.\xa0 keep caring for yourself and others as best as you can.\xa0 this is life.',
 'keep calm and buy iphone 8 keep calm and buy iphone 8 plus keep calm and buy iphone x. what is your favourite?']

In [0]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [0]:
def sent_TokenizeFunct(x):
    doc=nlp(x)
    #return doc
    tokens=[]
    for token in doc:
      if((token.is_stop==False and token.is_punct==False) and token.is_alpha==True):
        tokens.append(token.lemma_)
    return ' '.join(tokens)
    
sentenceTokenizeRDD = lowerCase_sentRDD.map(sent_TokenizeFunct)

In [79]:
sentenceTokenizeRDD.take(5)

['samsung',
 'thank gosh place watch have hd speed support hd',
 'happen home button iphone x',
 'power disease care cure care best life',
 'calm buy iphone calm buy iphone plus calm buy iphone favourite']

# Working on Videos

In [0]:
Videos1 =  spark.read.csv("/content/gdrive/My Drive/Colab Notebooks/GBvideos.csv", inferSchema=True, header=True)
Videos2 =  spark.read.csv("/content/gdrive/My Drive/Colab Notebooks/USvideos.csv", inferSchema=True, header=True)
videos = unionAll(Videos1,Videos2)

In [81]:
videos.show(5)

+-----------+--------------------+-----------------+-----------+--------------------+-------+-----+--------+-------------+--------------------+-----+
|   video_id|               title|    channel_title|category_id|                tags|  views|likes|dislikes|comment_total|      thumbnail_link| date|
+-----------+--------------------+-----------------+-----------+--------------------+-------+-----+--------+-------------+--------------------+-----+
|jt2OHQh0HoQ|Live Apple Event ...|      Apple Event|         28|apple events|appl...|7426393|78240|   13548|          705|https://i.ytimg.c...|13.09|
|AqokkXoa7uE|Holly and Phillip...|     This Morning|         24|this morning|inte...| 494203| 2651|    1309|            0|https://i.ytimg.c...|13.09|
|YPVcg45W0z4|My DNA Test Resul...|     emmablackery|         24|emmablackery|emma...| 142819|13119|     151|         1141|https://i.ytimg.c...|13.09|
|T_PuZBdT2iM|getting into a co...|            ProZD|          1|skit|korean|langu...|1580028|65729| 

In [82]:
videos_rdd = videos.select(['title']).rdd.flatMap(lambda x: x)
videos_rdd = videos_rdd.filter(lambda x: x is not None).filter(lambda x: x!='')
videos_rdd.take(5)

['Live Apple Event - Apple September Event 2017 - iPhone 8, iPhone X, iOS 11 - Apple Keynote',
 'Holly and Phillip Meet Samantha the Sex Robot | This Morning',
 "My DNA Test Results! I'm WHAT?!",
 "getting into a conversation in a language you don't actually speak that well",
 'Baby Name Challenge!']

In [0]:
header_Vid=videos_rdd.first()

In [0]:
data_rmv_col_vid = videos_rdd.filter(lambda row: row != header_Vid)

In [0]:
lowerCase_VidRDD = data_rmv_col_vid.map(lower)

In [86]:
lowerCase_VidRDD.take(5)

['holly and phillip meet samantha the sex robot | this morning',
 "my dna test results! i'm what?!",
 "getting into a conversation in a language you don't actually speak that well",
 'baby name challenge!',
 "revealed - fifa 18 stats for chelsea's hazard, luiz & christensen!"]

In [0]:
Vid_sentTokenizeRDD = lowerCase_VidRDD.map(sent_TokenizeFunct)

In [88]:
Vid_sentTokenizeRDD.take(5)

['holly phillip meet samantha sex robot morning',
 'dna test result',
 'get conversation language actually speak',
 'baby challenge',
 'reveal fifa stat chelsea hazard luiz christensen']

In [89]:
tags_rdd=videos.select(['tags']).rdd.flatMap(lambda x:x)
tags_rdd.take(5)

['apple events|apple event|iphone 8|iphone x|iphone 8 plus|iphone 7s|iphone 7s plus|ios 11',
 'this morning|interview|holly willoughby|phillip schofield|ruth langsford|eamonn holmes|chat shows - topic|chat show - topic|talk shows - topic|sex robot|samantha|real life',
 'emmablackery|emma blackery|emma|blackery|british vlogger|british youtuber|female vlogger|birdyboots|dna test|23andme|ancestry',
 "skit|korean|language|conversation|esl|japanese|foreign|communication|don't speak|struggle|foreigner|lotte mart|korea|south|weather|good|tutorial",
 'sprinkleofglitter|sprinkle of glitter|baby glitter|Louise Pentland|Louise Pentland Baby Name|Mrs Meldrum|Baby Names|Nice baby names|Unusual baby names']

In [0]:
header_tags = tags_rdd.first()

In [0]:
tags_rmv_col = tags_rdd.filter(lambda row: row != header_tags)

In [92]:
tags_rmv_col.take(5)

['this morning|interview|holly willoughby|phillip schofield|ruth langsford|eamonn holmes|chat shows - topic|chat show - topic|talk shows - topic|sex robot|samantha|real life',
 'emmablackery|emma blackery|emma|blackery|british vlogger|british youtuber|female vlogger|birdyboots|dna test|23andme|ancestry',
 "skit|korean|language|conversation|esl|japanese|foreign|communication|don't speak|struggle|foreigner|lotte mart|korea|south|weather|good|tutorial",
 'sprinkleofglitter|sprinkle of glitter|baby glitter|Louise Pentland|Louise Pentland Baby Name|Mrs Meldrum|Baby Names|Nice baby names|Unusual baby names',
 'Rio Ferdinand|Rio|Ferdinand|5 Magazine|5 Mag|#5|5 Rio Ferdinand|Rio Ferdinand interview|football|soccer|interview|Manchester United|Man United|fifa 18|fut 18|fut|fifa ratings|fifa 18 ratings|chelsea|chelsea fc|chelsea f.c|David Luiz|Eden Hazard|Christensen|Premier League|Champions League|Fifa18|Hazard Fifa 18|David Luiz Fifa 18|Christensen Fifa 18|Chelsea Fifa 18|Pack Opening|Fifa Pack

In [0]:
def split1(x):
  return (x.split("|"))

tags_rdd=tags_rmv_col.map(split1)

In [0]:
from pyspark.sql import Row
from pyspark.sql.functions import monotonically_increasing_id

row_title = Row("LemmetizeTitle") # Or some other column name
Title_DF = Vid_sentTokenizeRDD.map(row_title).toDF()
Title_DF = Title_DF.select("*").withColumn("id", monotonically_increasing_id())

row_tags=Row("Tokenized_Tags")
Tags_DF=tags_rdd.map(row_tags).toDF()
Tags_DF = Tags_DF.select("*").withColumn("id", monotonically_increasing_id())

In [95]:
Title_DF.show(5)

+--------------------+---+
|      LemmetizeTitle| id|
+--------------------+---+
|holly phillip mee...|  0|
|     dna test result|  1|
|get conversation ...|  2|
|      baby challenge|  3|
|reveal fifa stat ...|  4|
+--------------------+---+
only showing top 5 rows



In [96]:
Tags_DF.show(5)

+--------------------+---+
|      Tokenized_Tags| id|
+--------------------+---+
|[this morning, in...|  0|
|[emmablackery, em...|  1|
|[skit, korean, la...|  2|
|[sprinkleofglitte...|  3|
|[Rio Ferdinand, R...|  4|
+--------------------+---+
only showing top 5 rows



In [97]:
Tags_DF[['Tokenized_Tags']]

DataFrame[Tokenized_Tags: array<string>]

In [0]:
videos_DF = Title_DF.join(Tags_DF, on=['id'], how='left_outer')

In [99]:
videos_DF.show(5)

+----+--------------------+--------------------+
|  id|      LemmetizeTitle|      Tokenized_Tags|
+----+--------------------+--------------------+
|  26|testing soph x ma...|[grey hair, sophd...|
|  29|move vlog pink ho...|[moving vlog, ike...|
| 474|doom wolfenstein ...|[PC, IGN, PS4, Do...|
| 964|lloyd new lego ni...|[Google, YouTube,...|
|1677|project car ceo q...|[Project Cars 2 G...|
+----+--------------------+--------------------+
only showing top 5 rows



In [100]:
videos_DF.count()

15989

In [45]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer


tokenizer = Tokenizer(inputCol="LemmetizeTitle", outputCol="words")
wordsData = tokenizer.transform(videos_DF)

hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=5)
featurizedData = hashingTF.transform(wordsData)

idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)

rescaledData.select("id", "features").show()

+----+--------------------+
|  id|            features|
+----+--------------------+
|  26|(5,[0,1,2,3,4],[0...|
|  29|(5,[0,1,2,4],[0.3...|
| 474|(5,[0,1,2,3],[0.7...|
| 964|(5,[0,1,4],[1.419...|
|1677|(5,[0,1,2,3,4],[0...|
|1697|(5,[0,1,2,4],[0.3...|
|1806|(5,[1,2,3,4],[0.3...|
|1950|(5,[1,3,4],[0.750...|
|2040|(5,[0,1],[1.06440...|
|2214|(5,[3,4],[0.45814...|
|2250|(5,[0,1,4],[0.354...|
|2453|(5,[0,2,3],[0.354...|
|2509|(5,[1,3,4],[1.126...|
|2529|(5,[0,2,4],[2.128...|
|2927|(5,[0,1,3],[0.354...|
|3091|(5,[0,1,2,3,4],[0...|
|3506|(5,[0,1,2,3,4],[1...|
|3764|(5,[0,4],[0.70960...|
|4590|(5,[0,1,4],[0.709...|
|4823|(5,[0,3,4],[0.354...|
+----+--------------------+
only showing top 20 rows



In [0]:
documents=tags_rdd
from pyspark.mllib.feature import HashingTF, IDF
hashingTF = HashingTF()
tf = hashingTF.transform(documents)

In [0]:
  tf.cache()
  idf = IDF().fit(tf)
  tfidf = idf.transform(tf)

In [0]:
from pyspark.mllib.feature import Normalizer
labels = Tags_DF.select('id').rdd.flatMap(lambda x:x)
features = tfidf

normalizer = Normalizer()
data = labels.zip(normalizer.transform(features))

In [0]:
data.cartesian(data)\
    .map(lambda l: ((l[0][0], l[1][0]), l[0][1].dot(l[1][1])))\
    .sortByKey()\
    .collect()

In [0]:
# rdd = spark.sparkContext.parallelize([[1, "Delhi, Mumbai, Gandhinagar"],[2, " Delhi, Mandi"], [3, "Hyderbad, Jaipur"]])

# documents1 = rdd.map(lambda l: l[1].replace(" ", "").split(","))

# from pyspark.mllib.feature import HashingTF, IDF
# hashingTF = HashingTF()
# tf1 = hashingTF.transform(documents1)

In [0]:
    # tf.cache()
    # idf = IDF().fit(tf1)
    # tfidf = idf.transform(tf1)

In [0]:
# from pyspark.mllib.feature import Normalizer
# labels = rdd.map(lambda l: l[0])

In [105]:
# labels.take(5)

[1, 2, 3]

In [109]:
# Tags_DF.select('id').rdd.flatMap(lambda x:x).take(5)

[0, 1, 2, 3, 4]